In [1]:
ls data_creation/

__pycache__/  run_df_preprocess.py  wiki_parser.py


In [2]:
from data_creation import wiki_parser

In [ ]:
scipy.stats.

In [4]:
nltk

NameError: name 'nltk' is not defined